
# Planet Features API - RDMW

---

### Import Packages & Define Helper Functions

In [29]:
import os
import json
import requests
import time
import geopandas as gpd

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

### Authentication

Authentication with the Planet Features API be achieved using a valid Planet **API key**.

In [30]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = '98c3d498f425463ab3b3d3634bcef6b2'

    # construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')

### Our First Request

In [31]:
# Setup Planet Features API base URL
URL = "https://api.planet.com/features/v0/ogc/my/collections"

# Setup the session
session = requests.Session()
# Authenticate
session.auth = (API_KEY, "")

In [32]:
# Make a GET request to the Planet Features API
res = session.get(URL)
# Now we should get a response, hopefully it's a `200` code, saying everything is OK!
res.status_code

200

## Let's Create Our First Feature Collection

---

To upload an AOI Feature you must first create a Feature Collection. To create a Feature Collection make a POST request to the `/collections` endpoint providing a title and optionally a description.

The title will be used to create a "slug" that will be part of the collection's id.

A Collection identifier is a combination of optional user-supplied title and a hashid. For example, if a user creates a Collection with the title set to "My Great Places", the URL path identifier used to reference the Collection will be my-great-places-HASHID (where HASHID might be 4wr3wrz, for example). The Collection may also be referenced using only the HASHID - this way the user may change the title of their Collection if needed.


In [33]:
request = {
    "title" : "Waterbodies",
    "description" : "A collection of waterbodies for analysis"
}

# Send the POST request to the API stats endpoint
res = session.post(URL, json=request)

# Print response
p(res.json())

# let's save the collection-id to use later
collection_id = res.json()["id"]

{
  "id": "waterbodies-EZMJjv8",
  "title": "Waterbodies",
  "description": "A collection of waterbodies for analysis",
  "item_type": "feature",
  "links": [
    {
      "href": "https://api.planet.com/features/v0/ogc/my/collections/waterbodies-EZMJjv8",
      "rel": "self",
      "title": "This collection",
      "type": "application/json"
    },
    {
      "href": "https://api.planet.com/features/v0/ogc/my/collections/waterbodies-EZMJjv8/items",
      "rel": "features",
      "title": "Features",
      "type": "application/json"
    }
  ],
  "feature_count": 0,
  "area": null,
  "permissions": {
    "can_write": true,
    "shared_with_my_org": false,
    "is_owner": true
  }
}


## Adding Features to our Collection 
The features API guarantees that your geojson will work in Planet's Platform.

*Note* there is a 1500 vertex limit

In [34]:
# Load the shapefile
shapefile_path = 'c:/temp/planet/waterbody.shp'
shapefile_gdf = gpd.read_file(shapefile_path)
shapefile_wgs84  = shapefile_gdf.to_crs("epsg:4269")

# Get the bounding box coordinates
bounding_box = shapefile_wgs84.bounds.values[0]

print(bounding_box)

# Put your own Polygon here!
geo1 = {
    "type": "Polygon",
    "coordinates": [
                    [
                        [bounding_box[0], bounding_box[1]],
                        [bounding_box[0], bounding_box[3]],
                        [bounding_box[2], bounding_box[3]],
                        [bounding_box[2], bounding_box[1]],
                        [bounding_box[0], bounding_box[1]]
                    ]
                ]
            }

request = {
    "geometry": geo1,
    "id": "1", #optional, if you want to give your feature it's own id, use that here. this needs to be unique
    "properties" : {
        "title": "some-title",
        "description": "some-description"
    }

}

res = session.post(URL +"/"+ collection_id + "/items", json=request)

# Let's save the response as the short ref for later!
short_ref = res.json()[0]

print(short_ref)

[151.2994286  -27.75282169 151.31083805 -27.7450019 ]
pl:features/my/waterbodies-EZMJjv8/1-07przlm


## Getting/Editing Features

Once you've created features you can list all, or just get one. You can also edit a feature after you've created it, it will create a new version of that feature. 
View [the documentation to learn more](https://developers.planet.com/docs/apis/features/upload-and-validate/#accessing-your-features)

## Validating Features before Create

---

The Features API enables you to check if your Feature geometry is valid for use in Planet's Platform before uploading it to your Feature Collection. The `/validate` endpoint provides more details as to why a geometry is invalid, such as not meeting the vertex limit. The validation endpoint only supports validating Features.  
*i.e., you cannot validate a Feature Collection.*



In [35]:
validate_url =  'https://api.planet.com/features/v0/ogc/my/collections/validate'

# A request with no issues
request = {
    "type": "Feature", 
    "properties": {},
    "geometry": geo1,
    }


res = session.post(validate_url, json=request)
p(res.json())

{
  "status": "okay",
  "detail": "No errors."
}


### Let's try getting alternates for a feature with more than 1500 vertices

In [14]:
res = session.post(validate_url, json=request)
p(res.json())

{
  "status": "okay",
  "detail": "No errors."
}


### Get an alternative

In [15]:
alternate_url = 'https://api.planet.com/features/v0/ogc/my/collections/alternates'

res = session.post(alternate_url, json=request)
alternates = res.json()

# simplify the print here, we don't need to output all geometry points (cause it's a lot!)
for item in alternates["features"]:
    del item["geometry"]



p(alternates)

# if you want to see the whole output, comment the line below back in
# p(res.json())

{
  "type": "FeatureCollection",
  "features": [
    {
      "properties": {
        "technique": "Bounding box"
      }
    },
    {
      "properties": {
        "technique": "Convex hull"
      }
    },
    {
      "properties": {
        "technique": "Buffered by 1m and simplified"
      }
    },
    {
      "properties": {
        "technique": "Simplified to ~1cm"
      }
    }
  ]
}


*Note:* This endpoint provides provides recommendations and details about the technique used to alter the provided geometry so that it meets Planet's validity requirements.
Pick which geometry makes sense for you.

## Using the references 

Features references are supported in the Data API, Subscriptions API and Orders API.
We'll walk through some examples together below.

For all APIs you can replace your current geometry geojson block requests with a block like this:
```json
  "geometry": {
    "content": "pl:features/my/[collection]/[feature-id]", # your short ref
    "type": "ref" # instead of a Polygon or Multipolygon, the type is `ref`
  },

```

### Data API
#### Quick Searches with a Feature Reference

In [16]:
# Setup Item Types
item_types = ["PSScene"]


# Setup a geometry filter
geometry = {
    "type": "ref",
    "content": short_ref,
}

# Let's filter for the month of jan, and images that are not too cloudy.
filter = {
   "type":"AndFilter",
   "config":[
        {
            "type":"DateRangeFilter",
            "field_name":"acquired",
            "config":{
               "gte":"2022-12-20T00:00:00Z",
               "lte":"2023-12-19T00:00:00Z"
            },
        }, 
        {
           "type":"RangeFilter",
           "field_name":"cloud_cover",
           "config":{
              "lte":0.1
           },
        }
   ]
}


# Setup the request
request = {
    "item_types" : item_types,
    "geometry" : geometry,
    "filter": filter
    
}

In [17]:
# Send the POST request to the API quick search endpoint
quick_url =  "https://api.planet.com/data/v1/quick-search"
res = session.post(quick_url, json=request)


# Leaving the response commented out as it's quite long
# Print the response
# p(res.json())



Nice! You'll notice the geometry that was used to seach becomes hydrated in the response.


In [18]:
# save an item ID for the orders request below.
item_id = res.json()["features"][0]["id"]


#### Saved Searches with a Feature Reference

---

The `/searches` endpoint lets you created saved searches that can be reused.

To view your saved searches, visit the [`searches/?search_type=saved`](https://api.planet.com/data/v1/searches/?search_type=saved) endpoint.

Finally, let's create a saved search using a feature reference:

In [22]:
# Setup the saved searches endpoint url
searches_url = "https://api.planet.com/data/v1/searches"

In [24]:
# Setup the request, specifying a name for the saved search, along with the usual search item_types and filter.

request = {
    "name" : "Waterbodies search",
    "item_types" : item_types,
    "filter" : filter,
    "geometry": geometry
}

# Send a POST request to the saved searches endpoint (Create the saved search)
res = session.post(searches_url, json=request)

# Print the response
p(res.json())

{
  "__daily_email_enabled": false,
  "_links": {
    "_self": "https://api.planet.com/data/v1/searches/7b1e31d8b7a14f24a1c7a246d85c5ffb",
    "results": "https://api.planet.com/data/v1/searches/7b1e31d8b7a14f24a1c7a246d85c5ffb/results"
  },
  "created": "2024-03-12T03:34:21.369704Z",
  "filter": {
    "config": [
      {
        "config": {
          "gte": "2022-12-20T00:00:00Z",
          "lte": "2023-12-19T00:00:00Z"
        },
        "field_name": "acquired",
        "type": "DateRangeFilter"
      },
      {
        "config": {
          "lte": 0.1
        },
        "field_name": "cloud_cover",
        "type": "RangeFilter"
      }
    ],
    "type": "AndFilter"
  },
  "geometry": {
    "content": [
      "pl:features/my/waterbodies-YvwXoqR/1-mkaR1Ng#1"
    ],
    "type": "ref"
  },
  "id": "7b1e31d8b7a14f24a1c7a246d85c5ffb",
  "item_types": [
    "PSScene"
  ],
  "last_executed": null,
  "name": "Waterbodies search",
  "search_type": "saved",
  "updated": "2024-03-12T03:34:21.

*Note* you can see the reference returned in the search results

Now that we created a saved search, let's get a list of our saved searches:

In [25]:
# Send a GET request to the saved searches endpoint with the saved search type parameter (Get saved searches)
res = session.get(searches_url, params={"search_type" : "saved"})

p(res.json())


{
  "_links": {
    "_first": "https://api.planet.com/data/v1/searches/?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogImNyZWF0ZWQiLCAic29ydF9kZXNjIjogdHJ1ZSwgInNvcnRfc3RhcnQiOiBudWxsLCAic29ydF9sYXN0X2lkIjogbnVsbCwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjogeyJzZWFyY2hfdHlwZSI6ICJzYXZlZCJ9fQ%3D%3D",
    "_next": "https://api.planet.com/data/v1/searches/?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogImNyZWF0ZWQiLCAic29ydF9kZXNjIjogdHJ1ZSwgInNvcnRfc3RhcnQiOiAiMjAyNC0wMy0xMlQwMzozMzo1OS45MDIwNDdaIiwgInNvcnRfbGFzdF9pZCI6ICIyNzExMDEzODYwIiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjogeyJzZWFyY2hfdHlwZSI6ICJzYXZlZCJ9fQ%3D%3D",
    "_prev": "https://api.planet.com/data/v1/searches/?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogImNyZWF0ZWQiLCAic29ydF9kZXNjIjogdHJ1ZSwgInNvcnRfc3RhcnQiOiAiMjAyNC0wMy0xMlQwMzozNDoyMS4zNjk3MDRaIiwgInNvcnRfbGFzdF9pZCI6ICIyNzExMDEzOTgyIiwgInNvcnRfcHJldiI6IHRydWUsICJxdWVyeV9wYXJhbXMiOiB7InNlYXJjaF90eXBlIjogInNhdmVkIn19",
    "_self": "https://api.planet.com/da

And you'll see the reference returned when you query your saved searches.

### Orders API

In [26]:
orders_url = "https://api.planet.com/compute/ops/orders/v2"

In [27]:
request = {
    "name": "Test order with a reference",
    "source_type": "scenes",
    "products": [
        {
            "item_ids": [
                item_id # the id from the search request
            ],
            "item_type": "PSScene",
            "product_bundle": "analytic_udm2"
        }
    ],
    "tools": [
        {
            "clip": {
                "aoi": {
                    "content": short_ref,
                    "type": "ref"
                }
            }
        }
    ]

}

# Send a POST request to the orders endpoint
# WARNING: this will use your quota!
res = session.post(orders_url, json=request)

# Print the response
p(res.json())
# make the request

{
  "_links": {
    "_self": "https://api.planet.com/compute/ops/orders/v2/73cebb29-9361-4576-9534-d5745a32f036"
  },
  "created_on": "2024-03-12T03:34:53.547Z",
  "error_hints": [],
  "id": "73cebb29-9361-4576-9534-d5745a32f036",
  "last_message": "Preparing order",
  "last_modified": "2024-03-12T03:34:53.547Z",
  "name": "Test order with a reference",
  "products": [
    {
      "item_ids": [
        "20231118_001225_60_24fb"
      ],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
  ],
  "source_type": "scenes",
  "state": "queued",
  "tools": [
    {
      "clip": {
        "aoi": {
          "content": "pl:features/my/waterbodies-YvwXoqR/1-mkaR1Ng",
          "type": "ref"
        }
      }
    }
  ]
}


**Note** Checkout that AOI reference in the response!!

### Subscriptions API

In [28]:
# set up the subscriptions url
subs_url = "https://api.planet.com/subscriptions/v1"
    
# set up your bucket delivery
delivery_type = "your-delivery-type"
delivery_bucket = "your-bucket-name"
delivery_creds = "your-creds"

**Note** You can use the short_ref in the source geometry and optionally in the clip geometry 
If you choose to clip your order with a Features refence, it must be the same reference used in the source geometry.

In [24]:
# set up the subscription request

request = {
    "name": "Example Subscription",
    "source": {
        "type": "catalog",
        "parameters": {
            "geometry": {
                "content": short_ref,
                "type": "ref"
            },
            "time_range_type": "acquired",
            "start_time": "2024-01-01T00:00:00Z",
            "end_time": "2024-01-15T00:00:00Z",
            "item_types": [
                "PSScene"
            ],
            "asset_types": [
                "ortho_analytic_4b"
            ]
        }
    },
    "tools": [{
        "type": "clip",
        "parameters": {
            "aoi": {
                "content": short_ref,
                "type": "ref"
            }
        }
    }],
    "delivery": {
        "type": delivery_type,
        "parameters": {
            "bucket": delivery_bucket,
            "credentials": delivery_creds
        }
    }
}

# Send a POST request to the subscription endpoint
# WARNING: this will use your quota!
res = session.post(subs_url, json=request)

# Print the response
p(res.json())

{
  "name": "Example Subscription",
  "source": {
    "type": "catalog",
    "parameters": {
      "asset_types": [
        "ortho_analytic_4b"
      ],
      "item_types": [
        "PSScene"
      ],
      "geometry": {
        "coordinates": [
          [
            [
              -125.29632568359376,
              48.37084770238366
            ],
            [
              -125.29632568359376,
              49.335861591104106
            ],
            [
              -123.2391357421875,
              49.335861591104106
            ],
            [
              -123.2391357421875,
              48.37084770238366
            ],
            [
              -125.29632568359376,
              48.37084770238366
            ]
          ]
        ],
        "type": "Polygon"
      },
      "geom_ref": "pl:features/my/8vdGGZe/amXKR50#1",
      "start_time": "2024-01-01T00:00:00Z",
      "end_time": "2024-01-15T00:00:00Z",
      "publishing_stages": [
        "standard",
        "finali

**Note** You'll see the Features reference and the geojson returned in the subscriptions response.

## Conclusion 
**Congratulations!!**
You made it! That's how you can use and reuse geometry references across the Planet Platform. Go forth to use and reuse your geometry without needing to carry your geojsons around! 